In [34]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer  # this is used for filling the numeric missing value
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [17]:
df['fever'].value_counts()

fever
101.0    17
98.0     17
104.0    14
100.0    13
99.0     10
102.0    10
103.0     9
Name: count, dtype: int64

In [18]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [20]:
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [38]:
df.shape

(100, 6)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['has_covid']), df['has_covid'], test_size=0.2)

In [40]:
X_train.shape

(80, 5)

# Normal Working

# SimpleImputer

### SimpleImputer on Fever Column

In [41]:
si = SimpleImputer()

X_train_fever = si.fit_transform(X_train[['fever']])

X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

# Ordinal Encoding

### OrdinalEncoding on Cough

In [44]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])

X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

# OneHotEncoding

### OneHotEncoding on City and Gender

In [48]:
ohe = OneHotEncoder(drop ='first', dtype = np.int32, sparse_output = False)

## city and gender
X_train_city_gender = ohe.fit_transform(X_train[['city','gender']])

X_test_city_gender = ohe.fit_transform(X_test[['city','gender']])

X_train_city_gender.shape

(80, 4)

In [54]:
X_train_age = X_train.drop(columns=['gender','fever','city','cough']).values

X_test_age = X_test.drop(columns = ['gender','fever', 'cough','city']).values

X_train_age.shape

(80, 1)

# Concatenate

In [55]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_city_gender, X_train_cough), axis = 1)

X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_city_gender, X_test_cough), axis = 1)

X_train_transformed.shape

(80, 7)

# ColumnTransformer

In [56]:
from sklearn.compose import ColumnTransformer

In [57]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output= False, drop = 'first'), ['city','gender'])
], remainder='passthrough')

In [59]:
transformer.fit_transform(X_train).shape

(80, 7)

In [60]:
transformer.fit_transform(X_test).shape

(20, 7)